# Microsimulation Timing Tests

## Modules

In [1]:
import numpy as np
%load_ext cythonmagic

/Users/kfor/anaconda/lib/python2.7/site-packages/IPython/extensions/cythonmagic.py:21: UserWarning: The Cython magic has been moved to the Cython package
  warnings.warn("""The Cython magic has been moved to the Cython package""")


## Simulation Parameters

In [2]:
simulants = 1e6
prob_death = np.array([.01,.01,.02,.05,.1,.2,.2,.4,.7,1]) # array of probabilities of death for the 10 age groups, in order
ages = ['{0:g}-{1:g}'.format(i*10, i*10+9) for i in range(9)] + ['90 plus'] # names of the 10 age groups

## Python Loop

In [3]:
%%time
matrix = np.zeros((simulants,len(prob_death))) # rows=simulants, col=age groups
for age in range(prob_death.shape[0]):
    for sim in np.arange(simulants):
        if matrix[sim,age-1] == 1: # if the simulant has already died in a past age-time, they are dead forever
            matrix[sim,age] = 1
        else: # if they are not yet dead, expose them to the probability of death
            matrix[sim,age] = np.random.binomial(1,prob_death[age])
    print "Age: {age}\tPercent Dead: {dead:.1%}".format(age=ages[age], dead=np.mean(matrix[:,age]))

Age: 0-9	Percent Dead: 1.0%
Age: 10-19	Percent Dead: 2.0%
Age: 20-29	Percent Dead: 3.9%
Age: 30-39	Percent Dead: 8.7%
Age: 40-49	Percent Dead: 17.9%
Age: 50-59	Percent Dead: 34.2%
Age: 60-69	Percent Dead: 47.4%
Age: 70-79	Percent Dead: 68.5%
Age: 80-89	Percent Dead: 90.5%
Age: 90 plus	Percent Dead: 100.0%
CPU times: user 1min 16s, sys: 1.62 s, total: 1min 18s
Wall time: 1min 19s


## Python Vectorized

In [4]:
%%time
matrix = np.empty((simulants,len(prob_death)))
for age in range(len(prob_death)):
    matrix[:,age] = np.logical_or(np.random.binomial(1, prob_death[age], simulants), (matrix[:,age-1] if age > 0 else 0))
    print "Age: {age}\tPercent Dead: {dead:.1%}".format(age=ages[age], dead=np.mean(matrix[:,age]))

Age: 0-9	Percent Dead: 1.0%
Age: 10-19	Percent Dead: 2.0%
Age: 20-29	Percent Dead: 4.0%
Age: 30-39	Percent Dead: 8.8%
Age: 40-49	Percent Dead: 17.9%
Age: 50-59	Percent Dead: 34.4%
Age: 60-69	Percent Dead: 47.5%
Age: 70-79	Percent Dead: 68.4%
Age: 80-89	Percent Dead: 90.5%
Age: 90 plus	Percent Dead: 100.0%
CPU times: user 459 ms, sys: 47.9 ms, total: 507 ms
Wall time: 509 ms


## Cython

#### cython def

In [5]:
%%cython -l gsl -l gslcblas
cimport cython
from cython_gsl cimport *
import numpy as np
cimport numpy as np
from numpy cimport *
def csim(int simulants, ndarray[double, ndim=1] prob_death):
    cdef:
        gsl_rng *r = gsl_rng_alloc(gsl_rng_mt19937)
        Py_ssize_t i, j
        np.ndarray[uint32_t, ndim=2] M = np.empty((simulants,len(prob_death)), dtype='uint32')
    for i in range(simulants):
        for j in range(len(prob_death)):
            M[i,j] = 1 if j > 0 and M[i,j-1] else gsl_ran_bernoulli(r, prob_death[j])
    return M

#### run cython

In [6]:
%%time
pct_dead = np.mean(csim(simulants, prob_death), axis=0)
for a in range(len(ages)):
    print('Age: {age}\tPercent Dead: {dead:.1%}'.format(age=ages[a], dead=pct_dead[a]))

Age: 0-9	Percent Dead: 1.0%
Age: 10-19	Percent Dead: 2.0%
Age: 20-29	Percent Dead: 4.0%
Age: 30-39	Percent Dead: 8.7%
Age: 40-49	Percent Dead: 17.8%
Age: 50-59	Percent Dead: 34.3%
Age: 60-69	Percent Dead: 47.4%
Age: 70-79	Percent Dead: 68.4%
Age: 80-89	Percent Dead: 90.5%
Age: 90 plus	Percent Dead: 100.0%
CPU times: user 180 ms, sys: 17.8 ms, total: 198 ms
Wall time: 199 ms


## Theano

In [7]:
import theano
from theano import tensor as T
from theano.tensor.shared_randomstreams import RandomStreams

Couldn't import dot_parser, loading of dot files will not be possible.


In [8]:
T_rng = RandomStreams()
T_prob_alive = T.dvector()
T_simulants = T.iscalar()

In [9]:
T_matrix = T_rng.binomial(n=1, size=(T_simulants,T_prob_alive.shape[0]), p=T_prob_alive, ndim=2)
T_alive = T.extra_ops.cumprod(T_matrix, axis=1)
T_pct_dead = 1 - T_alive.mean(axis=0)
f = theano.function([T_simulants, T_prob_alive], [T_pct_dead])

In [10]:
%%time
pct_dead = f(simulants, 1-prob_death)
for a in range(len(ages)):
    print('Age: {age}\tPercent Dead: {dead:.1%}'.format(age=ages[a], dead=pct_dead[0][a]))

Age: 0-9	Percent Dead: 1.0%
Age: 10-19	Percent Dead: 2.0%
Age: 20-29	Percent Dead: 3.9%
Age: 30-39	Percent Dead: 8.8%
Age: 40-49	Percent Dead: 17.9%
Age: 50-59	Percent Dead: 34.3%
Age: 60-69	Percent Dead: 47.4%
Age: 70-79	Percent Dead: 68.5%
Age: 80-89	Percent Dead: 90.5%
Age: 90 plus	Percent Dead: 100.0%
CPU times: user 725 ms, sys: 64.3 ms, total: 789 ms
Wall time: 790 ms
